In [1]:
import os

import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.preprocessing.image import ImageDataGenerator

random_state = 42

# CIFAR10のデータをCNNでクラス分類②

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
y_train = np.eye(10)[y_train.astype('int32').flatten()]

x_test = x_test.astype('float32') / 255
y_test = np.eye(10)[y_test.astype('int32').flatten()]

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=10000)

In [3]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))  # 32x32x3 -> 28x28x6
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x6 -> 14x14x6
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))  # 14x14x6 -> 10x10x16
model.add(MaxPooling2D(pool_size=(2, 2)))  # 10x10x16 -> 5x5x16

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(120, activation='relu',
                kernel_initializer='he_normal'))  # 400 ->120
model.add(Dense(84, activation='relu', kernel_initializer='he_normal'))  # 120 ->84
model.add(Dense(10, activation='softmax'))  # 84 ->10

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,  # 3.1.1 左右にずらす
    height_shift_range=0.2,  # 3.1.2 上下にずらす
    horizontal_flip=True,  # 3.1.3 左右反転
    # 3.2.1 Global Contrast Normalization (GCN) (Falseに設定しているのでここでは使用していない)
    samplewise_center=False,
    samplewise_std_normalization=False,
    zca_whitening=False)  # 3.2.2 Zero-phase Component Analysis (ZCA) Whitening (Falseに設定しているのでここでは使用していない)

In [6]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                    steps_per_epoch=x_train.shape[0] // 100, epochs=30, validation_data=(x_valid, y_valid))

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
400/400 [==============================] - 39s 98ms/step - loss: 1.8749 - acc: 0.3036 - val_loss: 1.6114 - val_acc: 0.4019
Epoch 2/30
400/400 [==============================] - 38s 95ms/step - loss: 1.6387 - acc: 0.3985 - val_loss: 1.4939 - val_acc: 0.4537
Epoch 3/30
400/400 [==============================] - 43s 108ms/step - loss: 1.5759 - acc: 0.4226 - val_loss: 1.4061 - val_acc: 0.4891
Epoch 4/30
400/400 [==============================] - 42s 106ms/step - loss: 1.5221 - acc: 0.4464 - val_loss: 1.4183 - val_acc: 0.4918
Epoch 5/30
400/400 [==============================] - 38s 96ms/step - loss: 1.4801 - acc: 0.4645 - val_loss: 1.4052 - val_acc: 0.4927
Epoch 6/30
400/400 [==============================] - 45s 114ms/step - loss: 1.4381 - acc: 0.4778 - val_loss: 1.3077 - val_acc: 0.5246
Epoch 7/30
400/400 [==============================] - 48s 120ms/step - loss: 1.4060 - acc: 0.4893 - val_loss: 1.2960 - val_acc: 0.5415
Epoch 8/30

In [7]:
# 性能評価
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Test loss:  1.2251188915252686
Test accuracy:  0.5684


**Observation**
- sec2でのデータの水増しを行なっていない場合の性能が以下
    - Test loss:  1.265980617904663
    - Test accuracy:  0.553
- それよりは多少向上